# In this notebook we compare the performance of the Sinkhorn and the damped Newton algorithm.

In [ ]:
from __future__ import division
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
np.random.seed(1234)
%matplotlib inline
%load_ext autoreload
%autoreload 

In [ ]:
relative_path_to_new_folder = "../Images"
os.makedirs(relative_path_to_new_folder, exist_ok = True)
if not os.path.isdir('../Images/SinkhornvsDampedNewton_images'):
    os.makedirs('../Images/SinkhornvsDampedNewton_images')

In [ ]:
"""To compute distance matrix"""
def distmat( x, y ):
    return np.sum( x**2, 0 )[:,None] + np.sum( y**2, 0 )[None,:] - 2 * x.transpose().dot( y )

"""To Normalise a vector"""
normalize = lambda a: a/np.sum( a )

"""To Compute P"""
def GetP( u, K, v ):
    return u[:,None] * K *v[None,:]

def plotp( x, col, plt, scale = 200, edgecolors = "k"):
  return plt.scatter( x[0,:], x[1,:], s = scale, edgecolors = edgecolors,  c = col, cmap = 'plasma', linewidths = 2 )

In [ ]:
import computational_OT

## Entropy regularized formulation

The primal entropy regularized formulation of OT is given by:
$$
OT_{\varepsilon}(\alpha,\beta) = min_{\pi \in \mathcal{U}(\alpha,\beta)} \langle C,\pi \rangle +\varepsilon KL(\pi\|\alpha \otimes \beta)\ ,
$$
where
$\ 
KL(\pi\|\alpha \otimes \beta) 
\ $ is the KL-divergence and $\ \mathcal{U}(\alpha,\beta)=\{\pi: \pi\mathcal{1}=\alpha, \pi^{T}\mathcal{1}=\beta\}$.

# Sinkhorn iteration function
The optimal coupling $\pi^{*}$ has the following form :
$$
\pi^{*} = \alpha \odot diag(u)K diag(v)\odot \beta
$$
and we know that $\pi^{*}\mathbb{1}=\alpha$ and $(\pi^{*})^{T}\mathbb{1}=\beta$.
###
Therefore, Sinkhorn updates is given by the following alternative projections
$$
u^{t+1}  \leftarrow \frac{1}{K(v^{t}\odot \beta)}\ , \
v^{t+1}  \leftarrow \frac{1}{K^{T}(u^{t+1}\odot \alpha)}\ , 
$$
where 
$K = e^{-\frac{C}{\varepsilon}}\in M_{n\times m}(\mathbb{R}),\ \alpha \in \mathbb{R}^{n},\ \beta \in \mathbb{R}^{m}\ ,\ u\in \mathbb{R}^{n},\ v\in \mathbb{R}^{m}\ and \ (u^{0},v^{0})=(u,v)\ .$


In [ ]:
def sinkhorn( epsilons, N, x, y, iterations = 1000 ):
    print("Sinkhorn... ")
    print("Doing for (",N[0], N[1],").")
    SinkhornP                  = []
    results_Sinkhorn           = []
    times_Sinkhorn             = []
    #Cost matrix
    C = distmat( x, y )
    # a and b
    a = normalize( np.ones( N[0] ) )
    b = normalize( np.ones( N[1] ) )
    for eps in epsilons:
        print( "For epsilon = "+str(eps)+":" )    
        #Kernel
        K = np.exp( - C/eps )
        print( " |- Iterating")
        #Inflating
        u = a
        v = b
        start = time.time()
        Optimizer = computational_OT.sinkhorn(  K,
                                                a,
                                                b,
                                                u,
                                                v,
                                                eps )
        out = Optimizer._update( max_iterations = iterations )
        results_Sinkhorn.append( out )
        end = time.time()
        times_Sinkhorn.append( end - start )
        print( " |- Computing P" )
        print( "" )
        u_opt = np.exp( out['potential_f']/eps )
        K = np.exp( - C/eps )
        v_opt =  np.exp( out['potential_g']/eps )
        P_opt = GetP( u_opt, K, v_opt )
        SinkhornP.append( P_opt )
    # end for
    return {
        'results_list': results_Sinkhorn,
        'time_stamps' : times_Sinkhorn,
        'Ps'          : SinkhornP
    }
    

# Damped Newton(without preconditionING) iteration function

The dual formulation of OT is given by
$$
OT_{\varepsilon} = \max_{f\in \mathbb{R}^{n}, g\in\mathbb{R}^{m}} \langle f, \alpha \rangle + \langle g, \beta \rangle - \varepsilon\left(\langle\alpha \otimes \beta, e^{\frac{f}{\varepsilon}}\odot K \odot e^{\frac{g}{\varepsilon}}  \rangle-1\right)\ ,
$$
where
$$
\alpha \in \mathcal{M}_{1}(\mathcal{X}),\ \beta \in \mathcal{M}_{1}(\mathcal{Y}),\ \varepsilon>0,\ f\in\mathbb{R}^{n},\ g\in \mathbb{R}^{m}\ .
$$

The Hessian is given by 
$\nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g)=\frac{-1}{\varepsilon}
\begin{pmatrix}
\Delta(\alpha) && \pi_{\varepsilon}\\
\pi^{T}_{\varepsilon} && \Delta(\beta) 
\end{pmatrix}
\ , \ $ where $\pi\mathbb{1}_{m} = \alpha,\ \pi^{T}\mathbb{1}_{n}=\beta,\ $ and $\Delta = diag: \mathbb{R}^{n} \rightarrow M_{n}(\mathbb{R})$ is the linear operator mapping a vector  to a diagonal matrix  containing  this vector.


This implies 
$$
\begin{pmatrix}
\Delta(\alpha) && \pi_{\varepsilon}\\
\pi^{T}_{\varepsilon} && \Delta(\beta) 
\end{pmatrix}
\begin{pmatrix}
\mathbb{1}_{n}\\
\mathbb{1}_{m}
\end{pmatrix} = 0\ ,
$$
that is,
$$
\begin{pmatrix}
\mathbb{1}_{n}\\
\mathbb{1}_{m}
\end{pmatrix}\in \ker(\nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g))\ .
$$
Hence, $\nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g)$ is singular. Therefore, on regularization we have the following Hessian
$
H_{reg} := \nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g)+\lambda cc^{T}\ ,
$ 
where $c= \begin{pmatrix}\frac{\mathbb{1}}{\sqrt{n+m}}\\-\frac{\mathbb{1}}{\sqrt{n+m}}\end{pmatrix}\in M_{(n+m),1}(\mathbb{R})$.

Now, at the $k^{th}$ iteration solve
$\nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g)p_{k} = \nabla Q_{\alpha, \beta,\varepsilon}(f,g)$ to obtain the optimizing direction vector $p_{k}$ and then perform the Armijo condition to obtain the update step $\alpha_{k}$ such that we have the update
$$
(f,g) \leftarrow (f,g) + \alpha_{k} p_{k}\ .
$$

In [ ]:
def dampednewton( epsilons, N, x, y, rho = 0.95, c = 0.05, iterations = 50 ):
    print("Damped Newton... ")
    print( "Doing for (",N[0], N[1],")." )
    dampedNewtonP=[]
    results_dampedNewton  = []
    times_dampedNewton    = []
    Hessians_dampedNewton = []
    #Cost matrix
    C = distmat( x, y )
    # a and b
    a = normalize( np.ones( N[0] ) )
    b = normalize( np.ones( N[1] ) )
    for eps in epsilons:
        print("For epsilon = "+str(eps)+":")     
        #Kernel
        K = np.exp( - C/eps )
        f, g = a, b
        print( " |- Iterating")  
        start = time.time()
        Optimizer = computational_OT.damped_Newton( K,
                                                    a,
                                                    b,
                                                    f,
                                                    g,
                                                    eps,
                                                    rho,
                                                    c )
        out = Optimizer._update(    max_iterations = iterations,
                                    debug = False )
        end = time.time()
        if ( out != np.zeros(6) ).all():
            results_dampedNewton.append( out )
            times_dampedNewton.append( end - start )
            print( " |- Computing P" )
            print( "" )
            u_opt = np.exp( out['potential_f']/eps )
            K = np.exp( - C/eps )
            v_opt =  np.exp( out['potential_g']/eps )
            P_opt = GetP( u_opt, K, v_opt )
            dampedNewtonP.append( P_opt )
            print( " |- Recording (unstabilized) Hessian \n" )
            mat  = - eps * Optimizer.Hessian
            diag = 1/np.sqrt( np.concatenate( ( a, b ), axis = None ) )
            mat = diag[:,None] * mat * diag[None,:]
            Hessians_dampedNewton.append( mat )
        else:
            epsilons.remove( eps )
    # end for
    return {
        'results_list': results_dampedNewton,
        'time_stamps' : times_dampedNewton,
        'Ps'          : dampedNewtonP,
        'Hessians'    : Hessians_dampedNewton 
    }

# Damped Newton with preconditioning iteration function
Here we perform dual damped Newton with preconditioning. Here we consider $t$ eigenvalues of the Hessian that we want to move to one and form the following preconditioning matrix using the corresponding eigenvectors,
$$
P = \left(I_{n+m}-\sum_{i-1}^{t}\left(1 - \frac{1}{\sqrt{\lambda_{i}}}\right)y_{i}y_{i}^{T}\right)\ ,
$$
where
$$
y_{i} \in \ker\left(\nabla^{2}_{f}Q_{\alpha, \beta, \varepsilon}(f)-\lambda_{i}I_{n}\right),\ \forall i= 1,\dots,k\ ,
$$
 are orthonormal.

Now, at the $k^{th}$ iteration we solve the following equation:
$$
(P\nabla^{2}Q_{\alpha, \beta, \varepsilon}(f)P)(Pp_{k})=P\nabla Q_{\alpha, \beta, \varepsilon}(f)\ ,
$$
using iterative inversion methods such as "Conjugate gradient" and "GMRES" to get the ascent direction $p_{k}$, following which we use the Armijo condition to obtain the ascent step size $\alpha_{k}$.

In [ ]:
def dampednewtonprecondition( epsilons, N, x, y, null_vector, precond_vectors, rho = 0.95, c = 0.05,  iterations = 500, iter_inv = 30, optimizer = 'cg', version = None, debug = False):
    print("Damped Newton with preconditioning... ")
    print( "Doing for (",N[0], N[1],")." )
    reset_starting_point = True
    final_modified_Hessians = []
    dampedNewtonwithprecondP = []
    results_dampedNewtonwithprecond  = []
    times_dampedNewtonwithprecond    = []
    # Cost matrix
    C = distmat( x, y )
    # a and b
    a = normalize( np.ones(N[0]) )
    b = normalize( np.ones(N[1]) )
    f, g = None, None
    for eps in epsilons:
        print( "For epsilon = "+str(eps)+":" )    
        #Kernel
        K = np.exp( - C/eps )
        if (f is None) or (g is None): 
            f, g = a, b
        print( " |- Iterating" )  
        start = time.time()
        Optimizer = computational_OT.damped_Newton_with_preconditioning(    K,
                                                                            a,
                                                                            b,
                                                                            f,
                                                                            g,
                                                                            eps,
                                                                            rho,
                                                                            c,
                                                                            null_vector,
                                                                            precond_vectors[:] )
        out = Optimizer._update(    max_iterations = iterations,
                                    iterative_inversion = iter_inv,
                                    version = version,
                                    debug = debug,
                                    optType = optimizer )
        results_dampedNewtonwithprecond.append( out )
        end = time.time()
        times_dampedNewtonwithprecond.append( end - start )
        print( " |- Computing P" )
        print( "" )
        u_opt = np.exp( out['potential_f']/eps )
        K = np.exp( - C/eps )
        v_opt =  np.exp( out['potential_g']/eps )
        P_opt = GetP( u_opt, K, v_opt )
        dampedNewtonwithprecondP.append( P_opt )   
        if not reset_starting_point:
            f = Optimizer.x[:a.shape[0]]
            g = Optimizer.x[a.shape[0]:]
        final_modified_Hessians.append( Optimizer.modified_Hessian )
    # end for
    return {
        'results_list': results_dampedNewtonwithprecond,
        'time_stamps' : times_dampedNewtonwithprecond,
        'Ps'          : dampedNewtonwithprecondP,
        'Hessians'    : final_modified_Hessians 
    }    

# Comparison for Data size 400 and 500

In [ ]:
N = [ 400, 500 ]

In [ ]:
x     = np.random.rand( 2, N[0] ) - 0.5
theta = 2 * np.pi * np.random.rand( 1, N[1] )
r     = 0.8 + .2 * np.random.rand( 1, N[1] )
y     = np.vstack( ( r * np.cos(theta), r * np.sin(theta) ) )

## I. Sinkhorn

In [ ]:
epsilons = [  0.03, 0.001, 0.0009, 0.00084 ]
results_sinkhorn = sinkhorn( epsilons, N, x, y, iterations = 20000 )

### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )

plt.subplot( 2, 1, 1 ),
plt.title( "$||P1 -a||_1+||P1 -b||_1$" )
for i in range(len(results_sinkhorn['results_list'])):
  error = np.asarray( results_sinkhorn['results_list'][i]['error_a'] ) + np.asarray( results_sinkhorn['results_list'][i]['error_b'] ) 
  plt.plot( error, label = 'Sinkhorn for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.yscale( 'log' )
plt.legend()
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ConvergenceSinkhornvaryingepsilon.pdf", format = 'pdf' )
plt.show()

## II. Damped Newton without Preconditioning

In [ ]:
epsilons = [  0.5, 0.03, 0.02 ]
results_dampedNewton = dampednewton( epsilons, N, x, y )

### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range(len(results_dampedNewton['results_list'])):
  error = np.asarray( results_dampedNewton['results_list'][i]['error_a'] ) + np.asarray( results_dampedNewton['results_list'][i]['error_b'] )
  plt.plot( error, label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()
plt.yscale( 'log' )
plt.savefig("../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithoutPrecond.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

### Objective function plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "Objective Function" )
for i in range(len(results_dampedNewton['results_list'])):
  plt.plot( np.asarray( results_dampedNewton['results_list'][i]['objective_values'] ), label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Objective value" )
plt.legend()
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ObjectiveDampedNewtonwithoutPrecond.pdf", format = 'pdf' )
plt.show()


### Ascent step-size plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "Alpha" )
for i in range(len(results_dampedNewton['results_list'])):
  plt.plot( np.asarray( results_dampedNewton['results_list'][i]['linesearch_steps'] ), label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Alpha in log-scale" )
plt.legend()
plt.savefig( "../Images/SinkhornvsDampedNewton_images/AlphaDampedNewtonwithoutPrecond.pdf", format = 'pdf' )
plt.show()

### Plotting spectrum as a function of $\varepsilon$

In [ ]:
def spectral_decomposition( mat ):
    eig, v = np.linalg.eigh( mat )
    sorting_indices = np.argsort( eig )
    eig = eig[ sorting_indices ]
    v   = v[ : , sorting_indices ]
    print( "List of smallest eigenvalues: ", eig[ : 10 ] )
    print( "List of largest  eigenvalues: ", eig[ - 10 : ] )
    return eig, v

In [ ]:
eigs = []
eigvecs = []
for i in range(len(epsilons)):
    eps = epsilons[i]
    print("Spectral statistics of Hessian for epsilon = "+str(eps))
    ev = spectral_decomposition( results_dampedNewton['Hessians'][i] )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print("")
# end for

In [ ]:
fig, ax=plt.subplots( figsize = ( 5, 12 ), nrows = len(epsilons), ncols = 1, sharey = True )
plt.title("Histogram of eigenvalues.")
for i in range(len(epsilons)):
    ax[i].hist( eigs[i], 50)
    ax[i].set_title( " $\epsilon$: "+str(epsilons[i]))
    ax[i].set_xlabel( "Eigenvalues" )
    ax[i].set_yscale( "log" )
# end for
plt.subplots_adjust( wspace = 0, hspace = 0.5 )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/eigenhistunstabilizedDampedNewtonwithoutPrecond.pdf", format = 'pdf' )
plt.show()

### Actual preconditioning

In [ ]:
def build_preconditioners( num_eigs, modified_Hessian, ansatz = True ):
    # Diagonalize
    eigenvalues, eigenvectors = np.linalg.eigh( modified_Hessian )
    sorting_indices = np.argsort( eigenvalues )
    eigenvalues  = eigenvalues[ sorting_indices ]
    eigenvectors = eigenvectors[ : , sorting_indices ]
    # Form null vector
    if not ansatz:
        null_vector = eigenvectors[ : , 0 ]
    else:
        null_vector = np.hstack( ( np.ones(N[0]), - np.ones(N[1]) ) )
        norm = np.sqrt( N[0] + N[1] )
        null_vector = null_vector/norm
    # Form other vectors (only 13)
    _, m = eigenvectors.shape
    indices = []
    for i in range(num_eigs//2):
        indices.append( m - i - 2 )
        indices.append( i + 1 )
    # end fdr
    if num_eigs%2 != 0:
        indices.append( m - 1 - ( num_eigs//2 ) )
   
    precond_vectors = eigenvectors[ : , indices ]
    precond_vectors = []
    for index in indices:
        precond_vectors.append( eigenvectors[:,index] )
    # end for
    return null_vector, precond_vectors

In [ ]:
num_eigs = 13
null_vector, precond_vectors = build_preconditioners( num_eigs, results_dampedNewton['Hessians'][-1], ansatz = False )

## III. Damped Newton with Preconditioning

### Exact inversion

In [21]:
epsilons = [ 0.5, 0.001 , 0.00084 ] 
results_dampedNewtonwithprecond = dampednewtonprecondition( epsilons, N, x, y, null_vector, precond_vectors, iterations = 50, version = None, iter_inv = -1, rho = 0.95 )

#### Error plot

In [ ]:
plt.figure( figsize = ( 20,7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range(len(results_dampedNewtonwithprecond['results_list'])):
  error = np.asarray( results_dampedNewtonwithprecond['results_list'][i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond['results_list'][i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()  
plt.yscale( 'log' )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

#### Spectrum of the Hessian

In [ ]:
eigs = []
eigvecs = []
for i in range(len(epsilons)):
    eps = epsilons[i]
    print( "Spectral statistics of Hessian for epsilon = "+str(eps) )
    ev = spectral_decomposition( results_dampedNewtonwithprecond['Hessians'][i] )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print("")
# end for

In [ ]:
fig, ax=plt.subplots( figsize = ( 5, 12 ), nrows = len(epsilons), ncols = 1, sharey = True )
plt.title("Histogram of eigenvalues.")
for i in range(len(epsilons)):
    ax[i].hist( eigs[i], 50 )
    ax[i].set_title( " $\epsilon$: "+str(epsilons[i]))
    ax[i].set_xlabel( "Eigenvalues" )
    ax[i].set_yscale( "log" )
# end for
plt.subplots_adjust( wspace = 0, hspace = 0.3 )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/eigenhistunstabilizedDampedNewtonwithPrecond.pdf", format = 'pdf' )
plt.show()

### Iterative inversion = 2

#### rho = 0.5

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.001, 0.00084 ] 
results_dampedNewtonwithprecond = dampednewtonprecondition( epsilons, N, x, y, null_vector, precond_vectors, iterations = 500, iter_inv = 2, rho = 0.5 )

##### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range(len(results_dampedNewtonwithprecond['results_list'])):
  error = np.asarray( results_dampedNewtonwithprecond['results_list'][i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond['results_list'][i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()  
plt.yscale( 'log' )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_5itv2.pdf", format = 'pdf')
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

#### rho = 0.7

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_dampedNewtonwithprecond = dampednewtonprecondition( epsilons, N, x, y, null_vector, precond_vectors, iterations = 500, iter_inv = 2, rho = 0.7 )

##### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range(len(results_dampedNewtonwithprecond['results_list'])):
  error = np.asarray( results_dampedNewtonwithprecond['results_list'][i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond['results_list'][i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()  
plt.yscale( 'log' )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_7itv2.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

#### rho = 0.9

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_dampedNewtonwithprecond = dampednewtonprecondition( epsilons, N, x, y, null_vector, precond_vectors, iterations = 500, iter_inv = 2, rho = 0.9 )

##### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range(len(results_dampedNewtonwithprecond['results_list'])):
  error = np.asarray( results_dampedNewtonwithprecond['results_list'][i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond['results_list'][i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()  
plt.yscale( 'log' ) 
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_9itv2.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

### Iterative Inversion = 5

#### rho = 0.5

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_dampedNewtonwithprecond = dampednewtonprecondition( epsilons, N, x, y, null_vector, precond_vectors, iterations = 500, iter_inv = 5, rho = 0.5 )

##### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range(len(results_dampedNewtonwithprecond['results_list'])):
  error = np.asarray( results_dampedNewtonwithprecond['results_list'][i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond['results_list'][i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()  
plt.yscale( 'log' )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_5itv5.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

#### rho = 0.7

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_dampedNewtonwithprecond = dampednewtonprecondition( epsilons, N, x, y, null_vector, precond_vectors, iterations = 500, iter_inv = 5, rho = 0.7 )

##### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range(len(results_dampedNewtonwithprecond['results_list'])):
  error = np.asarray( results_dampedNewtonwithprecond['results_list'][i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond['results_list'][i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()  
plt.yscale( 'log' )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_7itv5.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

#### rho = 0.9

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_dampedNewtonwithprecond = dampednewtonprecondition( epsilons, N, x, y, null_vector, precond_vectors, iterations = 500, iter_inv = 5, rho = 0.9 )

##### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range(len(results_dampedNewtonwithprecond['results_list'])):
  error = np.asarray( results_dampedNewtonwithprecond['results_list'][i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond['results_list'][i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()  
plt.yscale( 'log' )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_9itv5.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

### Iterative Inversion = 10

#### rho = 0.5

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_dampedNewtonwithprecond = dampednewtonprecondition( epsilons, N, x, y, null_vector, precond_vectors, iterations = 500, iter_inv = 10, rho = 0.5 )

##### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range(len(results_dampedNewtonwithprecond['results_list'])):
  error = np.asarray( results_dampedNewtonwithprecond['results_list'][i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond['results_list'][i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()  
plt.yscale( 'log' )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_5itv10.pdf", format = 'pdf' ) 
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

#### rho = 0.7

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_dampedNewtonwithprecond = dampednewtonprecondition( epsilons, N, x, y, null_vector, precond_vectors, iterations = 500, iter_inv = 10, rho = 0.7 )

##### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range(len(results_dampedNewtonwithprecond['results_list'])):
  error = np.asarray(results_dampedNewtonwithprecond['results_list'][i]['error_a']) + np.asarray(results_dampedNewtonwithprecond['results_list'][i]['error_b'])
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()  
plt.yscale( 'log' )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_7itv10.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

#### rho = 0.9

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_dampedNewtonwithprecond = dampednewtonprecondition( epsilons, N, x, y, null_vector, precond_vectors, iterations = 500,iter_inv = 10, rho = 0.9 )

##### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range(len(results_dampedNewtonwithprecond['results_list'])):
  error = np.asarray( results_dampedNewtonwithprecond['results_list'][i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond['results_list'][i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" ) 
plt.ylabel( "Error in log-scale" )
plt.legend()  
plt.yscale( 'log' )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_9itv10.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" ) 

#### rho = 0.95

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_dampedNewtonwithprecond = dampednewtonprecondition(  epsilons, N, x, y, null_vector, precond_vectors, iterations = 500, iter_inv = 10, rho = 0.95 )

##### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range(len(results_dampedNewtonwithprecond['results_list'])):
  error = np.asarray( results_dampedNewtonwithprecond['results_list'][i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond['results_list'][i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()  
plt.yscale( 'log' )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_95itv10.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )